Notebook to prompt models to assist in information extraction of arguments from ToC.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

toc_df = pd.read_csv("/content/drive/MyDrive/LLM/extracted_briefs+toc.csv")
print(f"There are {len(toc_df)} rows in the dataset")

There are 14581 rows in the dataset


In [ ]:
# The first row is fucked for some reason so remove it
toc_df = toc_df.iloc[1:].reset_index(drop=True)

In [ ]:
# some additional cleaning after manual inspection of data

# toc_df['text'] = toc_df['text'].str.replace('â', "'", regex=False)
# toc_df['toc'] = toc_df['toc'].str.replace('â', "'", regex=False)
# toc_df['content'] = toc_df['content'].str.replace('â', "'", regex=False)
# Truncate strings in the 'toc' column to a maximum of n
toc_df['toc'] = toc_df['toc'].apply(lambda x: x if len(x) <= 4000 else x[:4000])
# del toc_df['merit_case']


In [ ]:
# toc_df.to_csv("/content/drive/MyDrive/LLM/extracted_briefs+toc_2.csv")

In [ ]:
print(toc_df.head)

<bound method NDFrame.head of        Unnamed: 0                                               text  \
0               1  b'No. 21-2\nIN THE\n\nSupreme Court of the Uni...   
1               2  b'No. 21-2\nIN THE\n\nSupreme Court of the Uni...   
2               3  b"No. _____\n\nIn the\nSupreme Court of the Un...   
3               4  b"No. _______\nIn The\n\nSupreme Court of the ...   
4               5  b'No. ___-______\n\nIn The\n\nSupreme Court of...   
...           ...                                                ...   
14575       14576  b"No.19-1355\nIN THE SUPREME COURT OF THE UNIT...   
14576       14577  b'No. _________\n=============================...   
14577       14578  b'No. _________\n\nIn the Supreme Court of the...   
14578       14579  b'No. 19-1359\n\nIN THE\n\n6upretne Court of t...   
14579       14580  b'No. 19â1360\n\nIn the Supreme Court of the...   

      created_timestamp downloaded_timestamp  \
0            08-02-2021           10-19-2021   
1        

Use this to find merit cases

In [ ]:
'''
merit_df = toc_df[~(toc_df['toc'].str.contains('certiorari', case=False, na=False )|
                   toc_df['text'].str.contains('amicus', case=False, na=False) |
                   toc_df['text'].str.contains('petition', case=False, na=False)
                   )]
print(len(merit_df))
'''

"\nmerit_df = toc_df[~(toc_df['toc'].str.contains('certiorari', case=False, na=False )|\n                   toc_df['text'].str.contains('amicus', case=False, na=False) |\n                   toc_df['text'].str.contains('petition', case=False, na=False)\n                   )]\nprint(len(merit_df))\n"

In [ ]:
# print(merit_df.iloc[100]['content'])

pile of law only has data from 2020 and 2021, which explains a lot.

In [ ]:
search_df = toc_df[toc_df['docket_num'] == '09-479']
print(len(search_df))

0


Apply more regex preprocessing to remove extra content. Hopefully this makes the extraction easier for the model later.

Truncating, and cleaning seems to work with the first example, try with a few others.

In [ ]:
import re
def clean_table_of_contents(toc_text):
    # Stage 1: Find the arguments
    # Remove standalone page numbers without a period. Must do this before removing periods below.
    toc_text = re.sub(r'^\s*\d+\s*$', '', toc_text, flags=re.MULTILINE)
    # Attempt to find the start of the arguments to extract. This will likely fail on some number of cases
    pattern = r'^(Arguments?|Reasons?\s+for).*\n?'
    matches = re.search(pattern, toc_text, re.MULTILINE | re.IGNORECASE)
    # print("Here are the matches")
    # print(matches)
    if not matches:
        # print("oopsie no matches")
        return None
    toc_text = toc_text[matches.end():]
    # print(f"After matching on Argument, the ToC looks like: \n{toc_text}")
    # Stage 2: Use re.search to find "CONCLUSION" on a line by itself, case-insensitive

    conclusion_match = re.search(r'^CONCLUSIONS?\b', toc_text, flags=re.MULTILINE | re.IGNORECASE) #remove ^ character?
    if conclusion_match:
        # If a match is found, slice after conclusion
        # print("Here is the match for conclusion")
        # print(conclusion_match)
        toc_text = toc_text[:conclusion_match.start()]
        # print(f"After matching on Conclusion, the ToC looks like: \n{toc_text}")

    # Now split on the periods
    split_text = re.split(r'\.\s*\.\s*\.\s*.*$', toc_text, flags=re.MULTILINE)
    # Removing empty strings and None elements that might result from capturing groups in the split

    # Finally, iterate through each line and apply other preprocessing steps, mainly removing periods and roman numerals
    processed_text = []
    for text in split_text:
      # text = re.sub(r'\.{2,}', ' ', text)  # Replace periods
      text = re.sub(r'\.\s*\.\s*\.\s*.*$', '', text, flags=re.MULTILINE) # Find any sequence of three periods, with any amount of space after them, and remove the rest of the line
      # text = re.sub(r'(\.\s){2,}.*$', '', text, flags=re.MULTILINE)

      # Remove lowercase Roman numerals at the end of lines, ensuring they're not part of section titles
      text = re.sub(r'^\s*(i{1,3}|iv|vi{0,3}|ix|xi{0,3}|xii{0,3}|xiii|xiv|xv)\s*[\.\s]*$', '', text, flags=re.MULTILINE)
      text = re.sub(r'\n+', ' ', text)
      text = text.strip()
      # Look for section indicators to remove any extra tokens at start of the line
      pattern = r'(I\.|II\.|III\.|IV\.|V\.|VI\.|VII\.|VIII\.|IX\.|X\.|1\.|2\.|3\.|4\.|5\.|6\.|7\.|8\.|9\.|10\.|A\.|B\.|C\.|D\.|E\.|F\.|G\.|H\.|I\.|J\.)'
      match = re.search(pattern, text)

      if match:
        text = text[match.start():]

      if text: # Ensure non-empty, non-whitespace only sections are kept
        processed_text.append(text)

    split_text = [s for s in split_text if s and s.strip()]
    return processed_text

In [ ]:
import re

def remove_conclusion(content_text):
  # Find all matches of "CONCLUSION" using re.finditer, which returns an iterator yielding match objects
  matches = list(re.finditer(r'^CONCLUSIONS?\b', content_text, flags=re.MULTILINE | re.IGNORECASE))

  if matches:
    # If matches are found, take the last match
    last_match = matches[-1]
    # Remove conclusion and everything after the last occurrence of "CONCLUSION"
    content_text = content_text[:last_match.start()]

  return content_text

In [ ]:
test_toc = toc_df.iloc[33]['toc']
print(test_toc)

TABLE OF CONTENTS
Table of Authorities .....................................................ii
Interest of Amici Curiae .............................................. 1
Summary of Argument ................................................ 1
Argument ..................................................................... 4
I.

The Doctrine of Forum Non Conveniens,
in its Application by Federal Courts, is
Marred by Inconsistency ................................ 4
A. The appellate courts disagree over the
deference due a plaintiff's forum choice .... 4
B. The defendant's burden has not been
consistently articulated or applied by
the appellate and district courts ................ 8
C. The lower courts disagree in their
weighing of public and private
factors and on the imposition of
dismissal conditions .................................... 9
D. Courts are divided with respect to
the choice of law analysis. ....................... 10

II. The Court Should Take this Opportunity to
Revisit the Do

In [ ]:
# Example input
toc_text = test_toc

# Clean the table of contents
cleaned_toc = clean_table_of_contents(toc_text)

In [ ]:
for i, line in enumerate(cleaned_toc):
  print(f"SECTION {i}")
  print(line)

SECTION 0
I. The Doctrine of Forum Non Conveniens, in its Application by Federal Courts, is Marred by Inconsistency
SECTION 1
A. The appellate courts disagree over the deference due a plaintiff's forum choice
SECTION 2
B. The defendant's burden has not been consistently articulated or applied by the appellate and district courts
SECTION 3
C. The lower courts disagree in their weighing of public and private factors and on the imposition of dismissal conditions
SECTION 4
D. Courts are divided with respect to the choice of law analysis
SECTION 5
II. The Court Should Take this Opportunity to Revisit the Doctrine
SECTION 6
A. Proposals for reform
SECTION 7
B. Proposals and grounds for abandonment


In [ ]:
test_toc = toc_df.iloc[10000]['toc']
print(test_toc)

TABLE OF CONTENTS
TABLE OF AUTHORITIES. . . . . . . . . . . . . . . . . . . iii
INTEREST OF AMICI CURIAE . . . . . . . . . . . . . . . . 1
INTRODUCTION AND SUMMARY
OF ARGUMENT . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
ARGUMENT . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
I.

STUDENT AMICI HAVE CHALLENGED A
VARIETY OF UNCONSTITUTIONAL
CAMPUS POLICIES ACROSS THE
NATION. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
A. Unconstitutional speech zones and
permitting policies may only generate
nominal damages rather than out-ofpocket costs that can be alleged or
recovered as damages . . . . . . . . . . . . . . . . 6
B. Some permit requirements also
necessitate payment of a âsecurity feeâ by
the student group hosting an event . . . . . 9
C. Unfettered discretion, leading to
constitutionally prohibited viewpoint
discrimination, is a common theme in
Student Amici challenges . . . . . . . . . . . . 10

II.

REMEDIES FOR NOMINAL DAMAGES
ARE IMPO

In [ ]:
test_toc = toc_df.iloc[10000]['toc']
print(test_toc)

# Example input
toc_text = test_toc

# Clean the table of contents
cleaned_toc = clean_table_of_contents(toc_text)

for i, line in enumerate(cleaned_toc):
  print(f"SECTION {i}")
  print(line)

TABLE OF CONTENTS
TABLE OF AUTHORITIES. . . . . . . . . . . . . . . . . . . iii
INTEREST OF AMICI CURIAE . . . . . . . . . . . . . . . . 1
INTRODUCTION AND SUMMARY
OF ARGUMENT . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
ARGUMENT . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
I.

STUDENT AMICI HAVE CHALLENGED A
VARIETY OF UNCONSTITUTIONAL
CAMPUS POLICIES ACROSS THE
NATION. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
A. Unconstitutional speech zones and
permitting policies may only generate
nominal damages rather than out-ofpocket costs that can be alleged or
recovered as damages . . . . . . . . . . . . . . . . 6
B. Some permit requirements also
necessitate payment of a âsecurity feeâ by
the student group hosting an event . . . . . 9
C. Unfettered discretion, leading to
constitutionally prohibited viewpoint
discrimination, is a common theme in
Student Amici challenges . . . . . . . . . . . . 10

II.

REMEDIES FOR NOMINAL DAMAGES
ARE IMPO

In [ ]:
regex_toc_df = toc_df['toc'].apply(clean_table_of_contents) # Apply regex to the whole column, save as new df to test

Look at some examples for how the regex worked

In [ ]:
idx = 3
print(toc_df.iloc[idx]['toc'])

TABLE OF CONTENTS
Page
QUESTION PRESENTED .......................................... i
PARTIES TO THE PROCEEDING ........................... ii
RULE 29.6 DISCLOSURE STATEMENT ................iii
RELATED PROCEEDINGS .....................................iii
TABLE OF CONTENTS............................................ iv
TABLE OF AUTHORITIES .................................... viii
OPINIONS BELOW ................................................... 2
JURISDICTION ......................................................... 2
CONSTITUTIONAL
AND
STATUTORY
PROVISIONS INVOLVED......................................... 3
INTRODUCTION ....................................................... 6
STATEMENT OF THE CASE ................................. 10
A.

Background ............................................... 10

B.

Constitutional objections .......................... 11

C.

Evidentiary Hearing on Motion to
Quash ........................................................ 13

D.

Trial Court Judg

In [ ]:
for thing in regex_toc_df.iloc[idx]:
  print(thing)

I. SCARBOROUGH'S COMMERCE CLAUSE ANALYSIS IS NO LONGER VIABLE AFTER LOPEZ
A. Courts and judges acknowledge that the two decisions are irreconcilable
B. The decision below is wrong
1. Scarborough's broad reading of the commerce power is fundamentally at odds with federalist principles
2. Section 922(g)'s possession element should require participation in, or a substantial effect on interstate commerce
3. The plain meaning of the nexus element of Â§ 922(g)'s possession crime does not support the prevailing interpretation
C. This case presents an ideal vehicle to overrule Scarborough
D. Federal overreach under Â§ 922(g) is an important and recurring issue
II. THE CIRCUITS ARE INTRACTABLY SPLIT OVER WHEN TO ALLOW A DEFENDANT TO PRESENT A JUSTIFICATION DEFENSE FOR ILLEGALLY POSSESSING A FIREARM
A. The circuits are divided
B. Two circuits allow the defense so long as the defendant has satisfied the common law elements of the defense
C. In other circuits, the failure to surrender the firearm 

In [ ]:
idx = 33
print(toc_df.iloc[idx]['toc'])

TABLE OF CONTENTS
Table of Authorities .....................................................ii
Interest of Amici Curiae .............................................. 1
Summary of Argument ................................................ 1
Argument ..................................................................... 4
I.

The Doctrine of Forum Non Conveniens,
in its Application by Federal Courts, is
Marred by Inconsistency ................................ 4
A. The appellate courts disagree over the
deference due a plaintiff's forum choice .... 4
B. The defendant's burden has not been
consistently articulated or applied by
the appellate and district courts ................ 8
C. The lower courts disagree in their
weighing of public and private
factors and on the imposition of
dismissal conditions .................................... 9
D. Courts are divided with respect to
the choice of law analysis. ....................... 10

II. The Court Should Take this Opportunity to
Revisit the Do

In [ ]:
for thing in regex_toc_df.iloc[idx]:
  print(thing)

I. The Doctrine of Forum Non Conveniens, in its Application by Federal Courts, is Marred by Inconsistency
A. The appellate courts disagree over the deference due a plaintiff's forum choice
B. The defendant's burden has not been consistently articulated or applied by the appellate and district courts
C. The lower courts disagree in their weighing of public and private factors and on the imposition of dismissal conditions
D. Courts are divided with respect to the choice of law analysis
II. The Court Should Take this Opportunity to Revisit the Doctrine
A. Proposals for reform
B. Proposals and grounds for abandonment


In [ ]:
idx = 10000
print(toc_df.iloc[idx]['toc'])

TABLE OF CONTENTS
TABLE OF AUTHORITIES. . . . . . . . . . . . . . . . . . . iii
INTEREST OF AMICI CURIAE . . . . . . . . . . . . . . . . 1
INTRODUCTION AND SUMMARY
OF ARGUMENT . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
ARGUMENT . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
I.

STUDENT AMICI HAVE CHALLENGED A
VARIETY OF UNCONSTITUTIONAL
CAMPUS POLICIES ACROSS THE
NATION. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
A. Unconstitutional speech zones and
permitting policies may only generate
nominal damages rather than out-ofpocket costs that can be alleged or
recovered as damages . . . . . . . . . . . . . . . . 6
B. Some permit requirements also
necessitate payment of a âsecurity feeâ by
the student group hosting an event . . . . . 9
C. Unfettered discretion, leading to
constitutionally prohibited viewpoint
discrimination, is a common theme in
Student Amici challenges . . . . . . . . . . . . 10

II.

REMEDIES FOR NOMINAL DAMAGES
ARE IMPO

In [ ]:
for thing in regex_toc_df.iloc[idx]:
  print(thing)

I. STUDENT AMICI HAVE CHALLENGED A VARIETY OF UNCONSTITUTIONAL CAMPUS POLICIES ACROSS THE NATION
A. Unconstitutional speech zones and permitting policies may only generate nominal damages rather than out-ofpocket costs that can be alleged or recovered as damages
B. Some permit requirements also necessitate payment of a âsecurity feeâ by the student group hosting an event
C. Unfettered discretion, leading to constitutionally prohibited viewpoint discrimination, is a common theme in Student Amici challenges
II. REMEDIES FOR NOMINAL DAMAGES ARE IMPORTANT TO FACILITATE REVISION OF UNCONSTITUTIONAL POLICIES ON CAMPUS AND DETER FUTURE VIOLATIONS
A. Nominal damages confer prevailing party status entitling the plaintiff to attorney feesâa powerful remedy to deter future violations
B. Remedies for completed constitutional violations expedite lasting policy changes that ensure free expression in the future, regardless of the type of amount of damage
C. Successful litigation may trigger leg

In [ ]:
idx = 12993
print(toc_df.iloc[idx]['toc'])

TABLE OF CONTENTS

Page

QUESTIONS PRESENTED ...................................

i

PARTIES TO THE PROCEEDING AND RULE
29.6 STATEMENT ...............................................

ii

RULE 14.3(b)(iii) STATEMENT ............................

iii

TABLE OF AUTHORITIES....................................

vii

OPINIONS BELOW ................................................

1

JURISDICTION .......................................................

1

CONSTITUTIONAL AND STATUTORY PROVISIONS INVOLVED ..........................................

1

STATEMENT OF THE CASE ................................

2

A. Factual Background ....................................

4

B. Proceedings Below.......................................

5

REASONS FOR GRANTING THE PETITION....

7

I. SCARBOROUGH'S COMMERCE
CLAUSE ANALYSIS IS NO LONGER
VIABLE AFTER LOPEZ................................

7

A. Courts and judges acknowledge that the
two decisions are irreconcilable .................

7

B. The decisio

In [ ]:
print(toc_df.iloc[idx]['url'])

http://www.supremecourt.gov/DocketPDF/20/20-6791/165363/20210104183852431_Penn%20Cert%20Petition%20Final.pdf


In [ ]:
for thing in regex_toc_df.iloc[idx]:
  print(thing)

I. SCARBOROUGH'S COMMERCE CLAUSE ANALYSIS IS NO LONGER VIABLE AFTER LOPEZ
A. Courts and judges acknowledge that the two decisions are irreconcilable
B. The decision below is wrong
1. Scarborough's broad reading of the commerce power is fundamentally at odds with federalist principles
2. Section 922(g)'s possession element should require participation in, or a substantial effect on interstate commerce
3. The plain meaning of the nexus element of Â§ 922(g)'s possession crime does not support the prevailing interpretation
C. This case presents an ideal vehicle to overrule Scarborough
D. Federal overreach under Â§ 922(g) is an important and recurring issue
II. THE CIRCUITS ARE INTRACTABLY SPLIT OVER WHEN TO ALLOW A DEFENDANT TO PRESENT A JUSTIFICATION DEFENSE FOR ILLEGALLY POSSESSING A FIREARM
A. The circuits are divided
B. Two circuits allow the defense so long as the defendant has satisfied the common law elements of the defense
C. In other circuits, the failure to surrender the firearm 

In [ ]:
# Apply regex to the whole column, save as new df to test
content_df = toc_df['content'].apply(lambda x: remove_conclusion(x) if pd.notnull(x) else x)

In [ ]:
idx = 33
print(print(toc_df.iloc[idx]['content']))

TABLE OF AUTHORITIES
Cases

Page(s)

Abad v. Bayer Corp.,
563 F.3d 663 (7th Cir. 2009) ................................ 7
AcuÃ±a-Atalaya v. Newmont Mining Corp.,
308 F. Supp. 3d 812 (D. Del. 2018) ...................... 3
American Dredging Co. v. Miller,
510 U.S. 443 (1994)............................................... 2
Archangel Diamond Corp. Liquidating Trust v.
Lukoil, 812 F.3d 799 (10th Cir. 2016) ................ 11
Ayco Farms, Inc. v. Ochoa,
862 F.3d 945 (9th Cir. 2017) ................................ 6
Burt v. Isthmus Development Co.,
218 F.2d 353 (5th Cir. 1955) ................................ 5
Capital Currency Exchange, N.V. v. National
Westminster Bank PLC,
155 F.3d 603 (2d Cir. 1998) ................................ 11
Carijano v. Occidental Petroleum Corp.,
643 F.3d 1216 (9th Cir. 2011)............................... 6
Delta Air Lines, Inc. v. Chimet, S.p.A.,
619 F.3d 288 (3d Cir. 2010) .................................. 8
DiFederico v. Marriott International Inc.,
714

In [ ]:
print(content_df.iloc[idx])

TABLE OF AUTHORITIES
Cases

Page(s)

Abad v. Bayer Corp.,
563 F.3d 663 (7th Cir. 2009) ................................ 7
AcuÃ±a-Atalaya v. Newmont Mining Corp.,
308 F. Supp. 3d 812 (D. Del. 2018) ...................... 3
American Dredging Co. v. Miller,
510 U.S. 443 (1994)............................................... 2
Archangel Diamond Corp. Liquidating Trust v.
Lukoil, 812 F.3d 799 (10th Cir. 2016) ................ 11
Ayco Farms, Inc. v. Ochoa,
862 F.3d 945 (9th Cir. 2017) ................................ 6
Burt v. Isthmus Development Co.,
218 F.2d 353 (5th Cir. 1955) ................................ 5
Capital Currency Exchange, N.V. v. National
Westminster Bank PLC,
155 F.3d 603 (2d Cir. 1998) ................................ 11
Carijano v. Occidental Petroleum Corp.,
643 F.3d 1216 (9th Cir. 2011)............................... 6
Delta Air Lines, Inc. v. Chimet, S.p.A.,
619 F.3d 288 (3d Cir. 2010) .................................. 8
DiFederico v. Marriott International Inc.,
714

Apply truncation and save the new df

In [ ]:
# toc_df['toc'] = toc_df['toc'].apply(clean_table_of_contents)

In [ ]:
# toc_df.to_csv("/content/drive/MyDrive/LLM/extracted_briefs+toc_3.csv")

Confirm the token length of the few shot prompt